In [1]:
import math
import random
from typing import List, Tuple, Any, Optional

# Deterministic RNG for reproducibility
random.seed(2025)

LANES = 8
LANE_PATTERN = [4, 5, 3, 6, 2, 7, 1, 8]

# Round sizes
ROUND_SIZES = {"R0": 80, "R1": 40, "R2": 24, "R3": 8}

# -------------------------------
# Domain objects
# -------------------------------
class Athlete:
    def __init__(self, athlete_id: int, name: str, base_ability: float):
        self.id = athlete_id
        self.name = name
        self.base_ability = base_ability  # lower is better
        self.last_round_time: Optional[float] = None

class Assignment:
    def __init__(self, round_name: str, heat_index: int, lane: int, athlete: Athlete, order_rank: int):
        self.round = round_name
        self.heat_index = heat_index
        self.lane = lane
        self.athlete = athlete
        self.order_rank = order_rank

class Result:
    def __init__(self, assignment: Assignment, time: float, place: int):
        self.assignment = assignment
        self.time = time
        self.place = place

# -------------------------------
# Generation and helpers
# -------------------------------
def generate_unseeded_athletes(n: int) -> List[Athlete]:
    athletes = []
    for i in range(n):
        ability = 10.8 + random.gauss(0, 0.6)
        ability = max(9.75, min(12.6, ability))
        athletes.append(Athlete(i + 1, f"Athlete_{i+1}", round(ability, 3)))
    return athletes

def serpentine_groups(entities: List[Any], size: int) -> List[List[Any]]:
    n = len(entities)
    heats = math.ceil(n / size)
    groups = [[] for _ in range(heats)]
    idx = 0
    direction = 1
    while idx < n:
        if direction == 1:
            for h in range(heats):
                if idx < n and len(groups[h]) < size:
                    groups[h].append(entities[idx]); idx += 1
            direction = -1
        else:
            for h in reversed(range(heats)):
                if idx < n and len(groups[h]) < size:
                    groups[h].append(entities[idx]); idx += 1
            direction = 1
    return groups

def assign_lanes(group: List[Athlete], round_name: str, heat_index: int, key_func) -> List[Assignment]:
    ranked = sorted(group, key=key_func)
    assignments = []
    for i, athlete in enumerate(ranked):
        lane = LANE_PATTERN[i] if i < len(LANE_PATTERN) else (i+1)
        assignments.append(Assignment(round_name, heat_index, lane, athlete, i + 1))
    return assignments

def simulate_heat(assignments: List[Assignment], noise_std: float = 0.08) -> List[Result]:
    results = []
    for a in assignments:
        noise = random.gauss(0, noise_std)
        simulated = max(9.5, a.athlete.base_ability + noise)
        results.append(Result(a, round(simulated, 3), place=0))
    # sort by time ascending; deterministic tie-break: athlete id
    results.sort(key=lambda r: (r.time, r.assignment.athlete.id))
    for i, r in enumerate(results, start=1):
        r.place = i
        r.assignment.athlete.last_round_time = r.time
    return results

def sort_with_tie_break(candidates: List[Tuple[Athlete, float]]) -> List[Tuple[Athlete, float]]:
    def key(x):
        ath, t = x
        prev = ath.last_round_time if ath.last_round_time is not None else float('inf')
        return (t, prev, ath.id)
    return sorted(candidates, key=key)

def progress_by_winners_and_time(all_heat_results: List[List[Result]], target_size: int) -> List[Athlete]:
    winners = [heat[0].assignment.athlete for heat in all_heat_results]
    remaining = []
    for heat in all_heat_results:
        for r in heat[1:]:
            remaining.append((r.assignment.athlete, r.time))
    remaining_sorted = sort_with_tie_break(remaining)
    need = target_size - len(winners)
    qualifiers = winners + [ath for ath, _ in remaining_sorted[:need]]
    # deduplicate while preserving order
    uniq, seen = [], set()
    for a in qualifiers:
        if a.id not in seen:
            uniq.append(a); seen.add(a.id)
    if len(uniq) != target_size:
        raise ValueError("Progression size mismatch; check qualification rule.")
    return uniq

# -------------------------------
# Lightweight FOL Ontology (kept, but facts are NOT printed)
# -------------------------------
class Predicate:
    def __init__(self, name: str, args: Tuple[Any, ...]):
        self.name = name
        self.args = args
    def __hash__(self): return hash((self.name, self.args))
    def __eq__(self, other): return isinstance(other, Predicate) and self.name == other.name and self.args == other.args
    def __repr__(self): return f"{self.name}({', '.join(map(str, self.args))})"

class Rule:
    def __init__(self, premises: List[Predicate], conclusion: Predicate):
        self.premises = premises
        self.conclusion = conclusion

class KnowledgeBase:
    def __init__(self):
        self.facts = set()
        self.rules = []
        self.categories = set()
        self.time_order = []

    def add_fact(self, pred: Predicate): self.facts.add(pred)
    def add_rule(self, rule: Rule): self.rules.append(rule)
    def add_category(self, name: str): self.categories.add(name)

    def query(self, name: str, *pattern) -> List[Tuple]:
        res = []
        for f in self.facts:
            if f.name == name and len(f.args) == len(pattern):
                match = True
                bindings = []
                for p, a in zip(pattern, f.args):
                    if isinstance(p, Var):
                        bindings.append(a)
                    elif p != a:
                        match = False; break
                if match:
                    res.append(tuple(bindings))
        return res

    def infer(self, max_iters=10):
        added = True
        iters = 0
        while added and iters < max_iters:
            added = False; iters += 1
            for rule in self.rules:
                if all(p in self.facts for p in rule.premises):
                    if rule.conclusion not in self.facts:
                        self.facts.add(rule.conclusion)
                        added = True

class Var:
    def __init__(self, name: str): self.name = name
    def __repr__(self): return f"?{self.name}"

def build_ontology(kb: KnowledgeBase,
                   round_name: str,
                   all_assignments: List[List[Assignment]],
                   all_results: List[List[Result]]):
    # Categories
    for cat in ["Athlete", "Event", "Heat", "Lane", "Assignment", "Result", "RaceRun", "QualificationDecision", "Round"]:
        kb.add_category(cat)
    kb.add_fact(Predicate("Round", (round_name,)))
    kb.add_fact(Predicate("Event", ("100m",)))

    for heat_idx, (assigns, results) in enumerate(zip(all_assignments, all_results), start=1):
        kb.add_fact(Predicate("Heat", (round_name, heat_idx)))
        kb.add_fact(Predicate("RaceRun", (round_name, heat_idx, f"time_{round_name}_{heat_idx}")))
        for a in assigns:
            kb.add_fact(Predicate("Athlete", (a.athlete.id,)))
            kb.add_fact(Predicate("Lane", (a.lane,)))
            kb.add_fact(Predicate("AssignedToLane", (round_name, heat_idx, a.athlete.id, a.lane)))
            kb.add_fact(Predicate("AssignedOrderRank", (round_name, heat_idx, a.athlete.id, a.order_rank)))
            kb.add_fact(Predicate("CompetesIn", (a.athlete.id, round_name, heat_idx)))
        for r in results:
            kb.add_fact(Predicate("Result", (round_name, heat_idx, r.assignment.athlete.id, r.time, r.place)))
            kb.add_fact(Predicate("RankInHeat", (round_name, heat_idx, r.assignment.athlete.id, r.place)))
            kb.add_fact(Predicate("HasRaceTime", (r.assignment.athlete.id, round_name, r.time)))
        kb.add_fact(Predicate("CentralLanePattern", (round_name, heat_idx, tuple(LANE_PATTERN))))

    # Winner -> AutoQualifies rule (materialized for known winners)
    for heat_idx, results in enumerate(all_results, start=1):
        winner = results[0].assignment.athlete.id
        kb.add_fact(Predicate("Winner", (round_name, heat_idx, winner)))
        kb.add_rule(Rule([Predicate("Winner", (round_name, heat_idx, winner))],
                         Predicate("AutoQualifies", (round_name, heat_idx, winner))))

def validate_capacity(kb: KnowledgeBase, round_name: str) -> bool:
    heats = set(h for (h,) in kb.query("Heat", round_name, Var("H")))
    for h in heats:
        assns = kb.query("AssignedToLane", round_name, h, Var("A"), Var("L"))
        if len(assns) > LANES: return False
        lanes = [l for (_, l) in assns]
        if len(set(lanes)) != len(lanes): return False
    return True

def validate_uniqueness(kb: KnowledgeBase, round_name: str) -> bool:
    athletes = set(a for (a,) in kb.query("Athlete", Var("A")))
    for a in athletes:
        heats = kb.query("CompetesIn", a, round_name, Var("H"))
        if len(heats) > 1: return False
    return True

def validate_centrality(kb: KnowledgeBase, round_name: str) -> bool:
    heats = set(h for (h,) in kb.query("Heat", round_name, Var("H")))
    for h in heats:
        ranks = kb.query("AssignedOrderRank", round_name, h, Var("A"), Var("R"))
        ranks.sort(key=lambda x: x[1])
        lanes = kb.query("AssignedToLane", round_name, h, Var("A"), Var("L"))
        lane_by_ath = {a: l for (a, l) in lanes}
        for i, (ath, r) in enumerate(ranks[:LANES]):
            if lane_by_ath.get(ath) != LANE_PATTERN[i]:
                return False
    return True

# -------------------------------
# Printing helpers (verbose competition log)
# -------------------------------
def print_round(all_assignments: List[List[Assignment]], all_results: List[List[Result]], round_name: str):
    for h_idx, (assigns, results) in enumerate(zip(all_assignments, all_results), start=1):
        print(f"\nRound {round_name} — Heat {h_idx}")
        print("  Assignments (seed/order_rank):")
        # Sort assignments by lane number for printing
        for a in sorted(assigns, key=lambda x: x.lane):
            print(f"    Lane {a.lane}: {a.athlete.name} (base {a.athlete.base_ability}s) rank {a.order_rank}")
        print("  Results (simulated times):")
        for place, r in enumerate(results, start=1):
            print(f"    {place}. {r.assignment.athlete.name} — {r.time}s (Lane {r.assignment.lane})")
    print()

def print_progression(from_round: str, to_round: str, qualifiers: List[Athlete]):
    print(f"\nProgression from {from_round} -> {to_round} (total qualifiers: {len(qualifiers)}):")
    for i, a in enumerate(qualifiers, start=1):
        print(f"  {i}. {a.name} — last_time={a.last_round_time}s base={a.base_ability}s")
    print()

# -------------------------------
# Round orchestration
# -------------------------------
def round_pipeline(athletes: List[Athlete], round_name: str, key_func, noise_std=0.08):
    groups = serpentine_groups(athletes, LANES)
    all_assignments, all_results = [], []
    for idx, group in enumerate(groups, start=1):
        assigns = assign_lanes(group, round_name, idx, key_func=key_func)
        results = simulate_heat(assigns, noise_std=noise_std)
        all_assignments.append(assigns)
        all_results.append(results)
    # print full heat-by-heat output for this round
    print_round(all_assignments, all_results, round_name)
    return all_assignments, all_results

def run_meet():
    # R0
    athletes_R0 = generate_unseeded_athletes(ROUND_SIZES["R0"])
    print("\n========== R0 (Unseeded) ==========")
    R0_assigns, R0_results = round_pipeline(athletes_R0, "R0", key_func=lambda a: a.base_ability, noise_std=0.085)

    # Progress to R1 (40)
    R1_field = progress_by_winners_and_time(R0_results, ROUND_SIZES["R1"])
    print_progression("R0", "R1", R1_field)

    # R1 seeded heats
    print("\n========== R1 (Seeded Heats) ==========")
    R1_assigns, R1_results = round_pipeline(R1_field, "R1", key_func=lambda a: a.base_ability, noise_std=0.075)

    # Progress to R2 (24)
    R2_field = progress_by_winners_and_time(R1_results, ROUND_SIZES["R2"])
    print_progression("R1", "R2", R2_field)

    # R2 semifinals
    print("\n========== R2 (Semifinals) ==========")
    R2_assigns, R2_results = round_pipeline(R2_field, "R2", key_func=lambda a: (a.last_round_time if a.last_round_time is not None else a.base_ability), noise_std=0.07)

    # Progress to R3 (8)
    R3_field = progress_by_winners_and_time(R2_results, ROUND_SIZES["R3"])
    print_progression("R2", "R3", R3_field)

    # R3 final
    print("\n========== R3 (Final) ==========")
    R3_assigns, R3_results = round_pipeline(R3_field, "R3", key_func=lambda a: (a.last_round_time if a.last_round_time is not None else a.base_ability), noise_std=0.065)

    # Build ontology and validate (no fact printing)
    for round_name, assigns, results in [
        ("R0", R0_assigns, R0_results),
        ("R1", R1_assigns, R1_results),
        ("R2", R2_assigns, R2_results),
        ("R3", R3_assigns, R3_results),
    ]:
        kb = KnowledgeBase()
        build_ontology(kb, round_name, assigns, results)
        kb.infer()
        assert validate_capacity(kb, round_name), f"Capacity violated in {round_name}"
        assert validate_uniqueness(kb, round_name), f"Uniqueness violated in {round_name}"
        assert validate_centrality(kb, round_name), f"Centrality violated in {round_name}"
        print(f"{round_name}: Ontology checks passed. Heats = {len(assigns)}")

    # Final results print (R3)
    print("\nFinal (R3) results:")
    for r in R3_results[0]:
        print(f"  {r.place}. {r.assignment.athlete.name} — {r.time}s (Lane {r.assignment.lane})")

if __name__ == "__main__":
    run_meet()


========== R0 (Unseeded) ==========

Round R0 — Heat 1
  Assignments (seed/order_rank):
    Lane 1: Athlete_20 (base 11.674s) rank 7
    Lane 2: Athlete_61 (base 10.559s) rank 5
    Lane 3: Athlete_80 (base 10.424s) rank 3
    Lane 4: Athlete_1 (base 9.992s) rank 1
    Lane 5: Athlete_40 (base 10.279s) rank 2
    Lane 6: Athlete_41 (base 10.507s) rank 4
    Lane 7: Athlete_60 (base 11.387s) rank 6
    Lane 8: Athlete_21 (base 12.215s) rank 8
  Results (simulated times):
    1. Athlete_1 — 10.005s (Lane 4)
    2. Athlete_80 — 10.32s (Lane 3)
    3. Athlete_40 — 10.324s (Lane 5)
    4. Athlete_41 — 10.546s (Lane 6)
    5. Athlete_61 — 10.565s (Lane 2)
    6. Athlete_60 — 11.322s (Lane 7)
    7. Athlete_20 — 11.786s (Lane 1)
    8. Athlete_21 — 12.223s (Lane 8)

Round R0 — Heat 2
  Assignments (seed/order_rank):
    Lane 1: Athlete_19 (base 11.15s) rank 7
    Lane 2: Athlete_22 (base 10.632s) rank 5
    Lane 3: Athlete_62 (base 10.383s) rank 3
    Lane 4: Athlete_39 (base 9.814s) rank 1
